# Import libraries

Description: this cell is used for importing the relevant libraries for the project.

In [ ]:
# Upload File using ipyfilechooser library
from ipyfilechooser import FileChooser
from ipywidgets import interact, interactive, fixed, interact_manual
from tqdm.notebook import tqdm, trange
# Video Player
from IPython.display import Video, display, clear_output
import time 
# Get the root directory of the project
from pyprojroot import here
# Copy File
import shutil
# Widget Packages
import ipywidgets as widgets
# In case widget extension not working
# jupyter nbextension enable --py widge|tsnbextension
import os
import sys
import re
from dotenv import load_dotenv, find_dotenv
load_dotenv()
os.environ['WANDB_API_KEY'] = os.getenv('WANDB_API_KEY')
global_path = here("")

Description: This cell is used for opening the file selector for user to select the video as input.

In [ ]:
def videoselectorinput():
    starting_directory = '../data/video'
    chooser = FileChooser(starting_directory)
    display(chooser)
    return chooser
def videoselectoroutputTSU():
    starting_directory = './video/output'
    chooser = FileChooser(starting_directory)
    display(chooser)
    return chooser

## Pipeline Selection
R6(Story): As a user, I want to create appropriate UI elements to allow for switching pipelines so that I can test the different models.

### Adding new pipeline
To add a new pipeline, add in the name of the pipeline in the options list in the dropdown widgets.

T05-157 (Story): As a user, I want to run the pipeline based on the dropdown selected. So that, I can integrate the pipeline dropdown into the notebook.

Description: This cell is used for setting and opening up the dropdown for the user to select the pipeline. After selecting and clicking "Confirm" the code will then run the selected pipeline.

In [ ]:
#print(os.getcwd())
pipelineList = ['TSU', 'STEP']
pipelineDropdown = widgets.Dropdown(

    options=pipelineList,
    value=pipelineList[0],
    description='Pipeline:')
# Function on what happen when confirm is been click.
def selectPipelineSet(b):
    if pipelineDropdown.value == "STEP":
        directory = os.getcwd()
        # Use Regex to match STEP
        if re.search(r"TSU", directory):
            %cd ..
            path =here("./NVIDIA-STEP-MODEL/STEP")
            %cd $path
            
        elif re.search(r"STEP", directory):
            pass
        else:
            path =here("./NVIDIA-STEP-MODEL/STEP")
            %cd $path
        print("Running " , pipelineDropdown.value, " pipeline")
        print("Loaded dependencies")
        #print("Please skip to Section 6 STEP Pipeline")
    elif pipelineDropdown.value == "TSU":
        directory = os.getcwd()
        # Use Regex to match STEP
        if re.search(r"TSU", directory):
            pass
        elif re.search(r"STEP", directory):
            %cd ../..
            path =here("./pipeline")
            %cd $path
        else:
            path =here("./pipeline")
            %cd $path
        print("Running " , pipelineDropdown.value, " pipeline")
        print("Loaded dependencies")
        path =here("./pipeline")
    else:
        pass
    
pipelineConfirm = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)

pipelineConfirm.on_click(selectPipelineSet)
pipelineBox = widgets.VBox([widgets.HBox([pipelineDropdown, pipelineConfirm])])
pipelineBox

<a href="#STEP">Click here to go to STEP section</a>
<br>
<a href="#TSU">Click here to go to TSU section</a>

# Data Exploration Section
R2 (Epic): As a user, I want a "Data Exploration" section in the notebook so that I can load and display video data from the TSU project.

## Video Upload / Choose using ipyfilechooser
R2 (Story): As a user, I want to upload/choose files from the data folder through an appropriate UI component (E.g. Browse files) in a notebook code cell so that I can pick and choose the video data I would like to process

Description: This cell is used for setting up the file choosers to select input video and to save output video.

In [ ]:
video = videoselectorinput()

## Upload selected video to the data folder (If needed)

Description: This cell is used for uploading the video to data folder.

In [ ]:
# Upload Function
# from pyprojroot import here
# import shutil
def upload(video):
    print(video.selected)
    source = video.selected
    # Source path
    # Destination path
    destination = (here("./data/video"))

    # Copy file from the selected path
    try:
        shutil.copy(source, destination)
        print("File copied successfully.")

    # If source and destination are same
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

    # If destination is a directory.
    except IsADirectoryError:
        print("Destination is a directory.")

    # If there is any permission issue
    except PermissionError:
        print("Permission denied.")

    # For other errors
    except:
        print("Error occurred while copying file.")

upload(video)

## Video Playback
R2 (Story): As a user, I want to see video playback of the chosen video file in an output cell so that I can check if it is the right video data I would like to process

Description: This cell is used for opening the file selector for user to select video input.

In [ ]:
# Select Video
video = videoselectorinput()

Description: This cell is used for running the input video. This is for the user to check if it is the same video they selected.

In [ ]:
full_path = video.selected
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

<a id='TSU'></a>
# Inference Section

R3 (Epic): As a user, I want to have an "Inference" section in the notebook so that I can perform inference using a pretrained HOI ML model based on the TSU project.

## Load a model
R3 (Story): As a user, I want to load a pre-trained or trained model using an appropriate UI component so that I can easily load the model for the inference section.

Description: This cell is used for setting up the dropdown list to allow the user to select the model to load.

In [ ]:
# Select model
modelList = [] 

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./models"): 
    if os.path.isdir(os.path.join("./models", x)) == False:
        if (x.endswith(".pkl")) == False:
            modelList.append(x)

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:')
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    print("Selected: " , modelDropdown.value)

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton])])
modelBox

## Choose Input video to load into TSU Project
R3 (Story): As a user, I want to choose an  input video files and other related input files, using an appropriate UI component, from the TSU project so that the system is able to pass the right files to the model.

Take Note: You only can select video that is in **testing subset** on the smarthome_CS_51.json file to run the inference video.

Description: This cell is used for selecting the video input to be passed to the TSU model.

In [ ]:
# Select Video
video = videoselectorinput()

## Run the model 
R3 (Story): As a user, I want to see inference results in the form of an output video with captions that indicate the current detected activity in each video frame so that I am able to see the inference results clearly on the screen.

Description: This cell is used for running the inference.py script based on the video and model selected.

In [ ]:
model = modelDropdown.value
loadmodel = './models/' + model
videoPath = video.selected
videoFile = video.selected_filename
print(videoFile)
print(videoPath)
%run -it inference.py  -input_video_file $videoFile -model $model  -load_model $loadmodel -video_path $videoPath
%wandb ict3104-team-5/inference-visualisation

## Output Video to view the inference

Description: This cell is used for print the filepath of the output video. Play the output video below the cell.

In [ ]:
videoFileName = video.selected_filename[:-4]
full_path = (os.path.join(here("./pipeline/video/output/"),f"{videoFileName}_caption.mp4"))
print(full_path)
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

# Feature Extraction Section

## Running main feature-extraction function
Description: This cell is used for running the main feature-extraction function to extract features from videos listed in video_paths.txt to create RGB .npy files for training later on.

In [ ]:
path = os.path.join(global_path , 'i3d-feature-extraction') 
%cd $path
%run -it main.py feature_type=i3d device="cuda:0" on_extraction=save_numpy streams=rgb output_path=../data/dataset/v_iashin_i3d/ file_with_video_paths=./sample/sample_video_paths.txt

## Running validate_train_test.py
Description: This cell is used for running validate_train_test.py to remove video IDs from smarthome_cs_51.json file and create an updated version called smarthome_cs_51_v2.json.

In [ ]:
!python validate_train_test.py

# Training Section
R4 (Epic): As a user, I want to create a "Training" section in the netbook so that I can train a HOI ML model based on the TSU project.

## Run_PDAN.sh for training
R4 (Story): As a user, I want to able to change the value for the argument in run_PDAN shell script with a UI so that I does not need to keep changing the value directly in the shell script.

R4 (Story): As a user, I can choose a dataset subfolder, using appropriate UI elements, from the data folder to use for the training so that I can select the data for training.

Description: This cell is used for form to fill up the parameters needed to run the run_PDAN.sh for training. Running the run_PDAN.sh script based on the parameters.

In [ ]:
title = widgets.HTML("<h1 style=\"text-align: center; \"><b>Run_PDAN</b></h1>")
label = widgets.HTML(
    value="<b>Label</b>\
    <br><b>Dataset Input:</b> Name of the Dataset to train\
    <br><b>Mode:</b> Inference Metric Methods: rgb or flow\
    <br><b>Spilt Setting:</b> Either CS or CV\
    <br><b>Model:</b> Select Model To Train\
    <br><b>Train:</b> Train or Test\
    <br><b>Num Channel:</b> Number of channel for image processing\
    <br><b>Learning Rate:</b> Learning rate indicates the step size that gradient descent takes towards local optima\
    <br><b>Kernel Size:</b> Size of a convolutional filter\
    <br><b>AP Type:</b> Metric in measuring the accuracy of object detectors: map or wap\
    <br><b>Epoch:</b> Number of iterations for the training process in Machine Learning\
    <br><b>Batch_Size:</b> Number of samples that is used in one epoch.\
    <br><b>Compute Info:</b> Information of the computation"
)
style = {'description_width': '90px'}

# Using Dropdown method
dataset_list = []
directoryDataset = os.path.join(global_path , 'data' , 'dataset') # Directory of the dataset (NPY files)
# Store the folder in the  dataset into dataset list
for x in os.listdir(directoryDataset):
    if os.path.isdir(os.path.join(directoryDataset, x)):
        # print(os.path.join(directoryDataset, x))
        dataset_list.append(x)

datasetDropDown = widgets.Dropdown(
    options=dataset_list,
    value=dataset_list[0],
    style=style,
    description='Dataset:')


mode_input = widgets.Text(
    value='rgb',
    placeholder='Enter Mode',
    description='Mode:',
    disabled=False,
    style=style,
    tooltip='Inference Metric Methods: rgb or flow'
    
)

split_input = widgets.Text(
    value='CS',
    placeholder='Enter Split Setting',
    description='Split Setting:',
    disabled=False,
    style=style,
    tooltip='Either CS or CV'
)

# Using Dropdown method
model_list = []
model_list.append("PDAN_TSU_RGB")
        
model_input = widgets.Dropdown(
    options=model_list,
    value=model_list[0],
    description='Model:',
    style=style,
    tooltip='Select Model to train.'
)


train_input = widgets.Checkbox(
    value=True,
    description = 'Train',
    disabled=False,
    indent=True,
    style=style,
    tooltip='Train or Test'
)

num_channel_input =  widgets.BoundedIntText(
    value=512,
    min=1,
    max=1000,
    step=1,
    description='Num Channel:',
    disabled=False,
    style=style,
    tooltip='Number of channel for image processing'
)

lr_input = widgets.FloatText(
    value=0.0002,
    description='Learning Rate:',
    disabled=False,
    style=style,
    tooltip='Learning rate indicates the step size that gradient descent takes towards local optima'
)

kernel_size_input =  widgets.BoundedIntText(
    value=3,
    min=1,
    max=5,
    step=1,
    description='Kernel Size:',
    disabled=False,
    style=style,
    tooltip='Size of a convolutional filter'
)

aptype_input = widgets.Text(
    value='map',
    placeholder='Enter APType',
    description='APType:',
    disabled=False,
    style=style,
    tooltip='Metric in measuring the accuracy of object detectors: map or wap'
)

epoch_input = widgets.BoundedIntText(
    value=140,
    min=1,
    max=1000,
    step=1,
    description='Epoch:',
    disabled=False,
    style=style,
    tooltip='Number of iterations for the training process in Machine Learning'
)

batch_size_input = widgets.Dropdown(
    options=['1', '2', '4', '8', '16', '32', '64', '128', '256', '512', '1024'],
    value='1',
    description='Batch_Size:',
    disabled=False,
    tooltip='Number of samples that is used in one epoch.',
    style=style
    
)


comp_info_input = widgets.Text(
    value='TSU_CS_RGB_PDAN',
    placeholder='Enter Compute Info',
    description='Compute Info:',
    disabled=False,
    style=style,
    tooltip='Information of the computation.'
)

button = widgets.Button(
    description='Save',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save Pref',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    style=style
)

jsonDirectory = os.path.join(directoryDataset , 'JSON')
jsonFiles = []
for x in os.listdir(jsonDirectory):
    if os.path.isdir(os.path.join(jsonDirectory, x)) == False:
        jsonFiles.append(x)

json_file_input = widgets.Dropdown(
    options=jsonFiles,
    value=jsonFiles[0],
    style=style,
    description='Json File:')
    

def selectWidgetSet(b):
    print("Selected Dataset: " , datasetDropDown.value)
    print("Selected Mode: " , mode_input.value)
    print("Selected Split Setting: " , split_input.value)
    print("Selected Model: " , model_input.value)
    print("Selected Train: " , train_input.value)
    print("Selected Num Channel: " , num_channel_input.value)
    print("Selected Learning Rate: " , lr_input.value)
    print("Selected Kernel Size: " , kernel_size_input.value)
    print("Selected APType: ", aptype_input.value)
    print("Selected Epoch: ", epoch_input.value)
    print("Selected Batch: " , batch_size_input.value)
    print("Selected Compute Info: " , comp_info_input.value)
    dataset_path = os.path.join("../data/dataset/" , datasetDropDown.value)
    print("Dataset Path: ", dataset_path)
    json_path = os.path.join("../data/dataset/JSON/" , json_file_input.value)
    print("Json Path: ", json_path)
    %run -i train.py -dataset $datasetDropDown.value -mode $mode_input.value -split_setting $split_input.value -model $model_input.value -train $train_input.value -num_channel $num_channel_input.value -lr $lr_input.value -kernelsize $kernel_size_input.value -APtype $aptype_input.value -epoch $epoch_input.value -batch_size $batch_size_input.value -comp_info $comp_info_input.value -rgb_root $dataset_path -json_file $json_path
    %wandb ict3104-team-5/training-visualisation

display(title, label, datasetDropDown, mode_input,split_input, model_input, train_input, num_channel_input, lr_input, kernel_size_input, aptype_input, epoch_input, batch_size_input, comp_info_input, json_file_input ,button)
button.on_click(selectWidgetSet) 

# Testing Section
R5 (Epic): As a user, I want to have a "Testing" section in the notebook so that I can evaluate a trained model based on the TSU project.

R5 (Story): As a user, I want to load a pretrained model using an appropriate UI component so that I can easily choose the type of pretrained model I would like to process the data with.

Description: This cell is used for dropdown for the user to select the model to be tested

In [ ]:
# Select model
modelList = [] 

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./models"): 
    if os.path.isdir(os.path.join("./models", x)) == False:
        if (x.endswith(".pkl")):
            modelList.append(x)

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:')
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    print("Selected: " , modelDropdown.value)

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton])])
modelBox

R5 (Epic): As a user, I want to have a "Testing" section in the notebook so that I can evaluate a trained model based on the TSU project.

In [ ]:
%run -i test.py -split CS -pkl_path "models/"$modelDropdown.value -batch_size batch_size_input.value -learning_rate lr_input.value -epochs epoch_input.value
%wandb ict3104-team-5/testing-visualisation 

# Inference Section for Training model

R3 (Epic): As a user, I want to have an "Inference" section in the notebook so that I can perform inference using a pretrained HOI ML model based on the TSU project.

In [ ]:
# Select model
modelList = [] 

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./models"): 
    if os.path.isdir(os.path.join("./models", x)) == False:
        if (x.endswith(".pkl")) == False:
            modelList.append(x)

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:')
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    print("Selected: " , modelDropdown.value)
    model = modelDropdown.value
    loadmodel = './models/' + model
    videoPath = video.selected
    videoFile = video.selected_filename
    print(videoFile)
    print(videoPath)
    %run -it inference.py  -input_video_file $videoFile -model $model  -load_model $loadmodel -video_path $videoPath
    %wandb ict3104-team-5/inference-visualisation

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton])])
modelBox

# Output Video for Inference Section for Training model

In [ ]:
videoFileName = video.selected_filename[:-4]
full_path = (os.path.join(here("./pipeline/video/output/"),f"{videoFileName}_caption.mp4"))
print(full_path)
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

# NVIDIA STEP Section
R6 (Epic): As a user, I want to able to configure the notebook using appropriate UI elements coupled with the right .py modules so that R2-5 can be performed based on another pipeline, e.g., the NVIDIA STEP pipeline 

<a id='STEP'></a>
## Nvidia Step Pipeline
R6(Story): As a user, I want to ensure selected pipeline's dependencies are changed to ensure the right dependencies are given to the appropriate models so that the selected model will be run

Description: This cell is used for setup the model

In [ ]:
#Else run this
%run -i setup.py build develop

T05-128: As a User, I want pre-process the input video into frames for Nvidia STEP model, So that I can run the model successfully.

T05-127 As a User, I want the output of Nvidia STEP model to be in a video format, So that I can easily view the results of the model.

T05-119: As a user, I want to have multiple dropdowns to select the input (1) dataset (2) video so that I can select the dataset videos easily

Description: This cell is used for setting and  opening up the dropdowns to allow user to select the dataset and the video input.

In [ ]:
# Select video
video_list = [] 
dataset_list = [] 
directoryVideo = "./Input/" # Directory of the video (.mp4)
directoryDataset = "./datasets/demo/frames/" # Directory of the dataset (NPY files)

# Store the folder in the  dataset into dataset list
for x in os.listdir(directoryDataset):
    if os.path.isdir(os.path.join(directoryDataset, x)):
        # print(os.path.join(directoryDataset, x))
        dataset_list.append(x)

# Store the video names into video list
for x in os.listdir(directoryVideo):
    if x.endswith(".mp4"):
        video_list.append(x)

datasetDropDown = widgets.Dropdown(
    options=dataset_list,
    description='Dataset:')

videoDropDown = widgets.Dropdown(
    options=video_list,
    description='Video: ')

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
# Function on what happen when confirm is been click. To be intergrated with some other code
def selectTrimSet(b):
    print("Selected Dataset: " , datasetDropDown.value)
    print("Selected Video: " , videoDropDown.value)

confirmButton.on_click(selectTrimSet)
display(datasetDropDown, videoDropDown, confirmButton)

Description: This cell is used to caption every frame of the video. Afterwards, it will run the STEP Model.

In [ ]:
import cv2
import numpy as np
import glob

cap= cv2.VideoCapture('Input/' + videoDropDown.value)
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('datasets/demo/frames/' + datasetDropDown.value + '/frame'+str(i).zfill(4)+'.jpg',frame)
    i+=1

cap.release()
cv2.destroyAllWindows()

## Traubung code will be here
%run -i demo.py

Description: This cell is used for reading every frames in the results folder.It will write the captions of the frames onto the output video.

In [ ]:
import cv2
import numpy as np
import glob
img_array = []
for filename in glob.glob('datasets/demo/results/' + datasetDropDown.value + '/*.jpg'):
    img = cv2.imread(filename) 
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)


out = cv2.VideoWriter('Output/demo-result.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

## Training

In [ ]:
import torch
import cv2
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

In [ ]:
%run ./scripts/generate_label.py "datasets/ava/label/ava_train_v2.1.csv" 
%run ./scripts/generate_label.py "datasets/ava/label/ava_val_v2.1.csv"

In [ ]:
%run ./scripts/extract_clips.py

In [ ]:
data_root="datasets/ava/"
save_root="datasets/ava/cache/"
pretrain_path="pretrained/ava_step.pth"

name="STEP"
base_net="i3d"
det_net="two_branch"
resume_path="Auto"

T=3
max_iter=3    # index starts from 1
iterative_mode="temporal"
anchor_mode="1"
temporal_mode="predict"
pool_mode="align"
pool_size=7

# training schedule
num_workers=2
max_epochs=144

batch_size=1
optimizer="adam"
base_lr=7.5e-5
det_lr0=1.5e-4
det_lr=7.5e-4
save_step=11465
print_step=500
scheduler="cosine"
milestones="-1"
warmup_iters=1000

# losses
dropout=0.3
fc_dim=256
lambda_reg=5
lambda_neighbor=1
cls_thresh="0.2,0.35,0.5"
reg_thresh="0.2,0.35,0.5"
max_pos_num=5
neg_ratio=2
NUM_SAMPLE=-1
topk=300
evaluate_topk=300

# data augmentation / normalization
scale_norm=2    # for i3d
do_flip="True"
do_crop="True"
do_photometric="True"
do_erase="True"
freeze_affine="True"
freeze_stats="True"


%run train.py --data_root $data_root --save_root $save_root \
    --name $name --pretrain_path $pretrain_path --resume_path $resume_path \
    --base_net $base_net --det_net $det_net --max_iter $max_iter --T $T \
    --iterative_mode $iterative_mode --anchor_mode $anchor_mode --anchor_mode $anchor_mode --temporal_mode $temporal_mode \
    --pool_mode $pool_mode --pool_size $pool_size --save_step $save_step --topk $topk --evaluate_topk $evaluate_topk \
    --num_workers $num_workers --max_epochs $max_epochs --batch_size $batch_size --print_step $print_step \
    --optimizer $optimizer --base_lr $base_lr --det_lr $det_lr --det_lr0 $det_lr0 --milestones $milestones \
    --scale_norm $scale_norm --do_flip $do_flip --do_crop $do_crop --do_photometric $do_photometric --do_erase $do_erase \
    --fc_dim $fc_dim --dropout $dropout --NUM_SAMPLE $NUM_SAMPLE --scheduler $scheduler --warmup_iters $warmup_iters \
    --cls_thresh $cls_thresh --reg_thresh $reg_thresh --max_pos_num $max_pos_num --neg_ratio $neg_ratio \
    --freeze_affine $freeze_affine --freeze_stats $freeze_stats --lambda_reg $lambda_reg --lambda_neighbor $lambda_neighbor

In [ ]:
data_root="datasets/ava/" #pickle name
save_root="datasets/ava/cache/" #
kinetics_pretrain="pretrained/i3d_kinetics.pth"

name="Cls"
base_net="i3d"
det_net="two_branch"
resume_path="Auto"

T=9
max_iter=1    # index starts from 1
iterative_mode="spatial"
pool_mode="align"
pool_size=7

# training schedule
num_workers=1
max_epochs=14
batch_size=1
optimizer="adam"
base_lr=5e-5
det_lr0=1e-4
det_lr=5e-4
save_step=22930
print_step=2000
scheduler="cosine"
milestones="-1"
warmup_iters=1000

# losses
dropout=0.3
fc_dim=256

# data augmentation / normalization
scale_norm=2    # for i3d
do_flip="True"
do_crop="True"
do_photometric="True"
do_erase="True"
freeze_affine="True"
freeze_stats="True"


%run train_cls.py --data_root $data_root --save_root $save_root \
    --name $name --resume_path $resume_path --kinetics_pretrain $kinetics_pretrain \
    --base_net $base_net --det_net $det_net --max_iter $max_iter --T $T \
    --iterative_mode $iterative_mode \
    --pool_mode $pool_mode --pool_size $pool_size --save_step $save_step \
    --num_workers $num_workers --max_epochs $max_epochs --batch_size $batch_size --print_step $print_step \
    --optimizer $optimizer --base_lr $base_lr --det_lr $det_lr --det_lr0 $det_lr0 --milestones $milestones \
    --scale_norm $scale_norm --do_flip $do_flip --do_crop $do_crop --do_photometric $do_photometric --do_erase $do_erase \
    --fc_dim $fc_dim --dropout $dropout  --scheduler $scheduler --warmup_iters $warmup_iters \
    --freeze_affine $freeze_affine --freeze_stats $freeze_stats

## STEP Testing

In [ ]:
%run test.py